# Feature engineering regresion logistica

In [115]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [116]:
df = pd.read_csv("../data/interim/df_features_general.csv")

## Eliminar variables no predictivas

In [117]:
# Eliminar columnas innecesarias o no predictivas
cols_a_eliminar = [
    "Unnamed: 0", "user_id", "total_spend", "monto_promedio_mensual.x", "monto_promedio_transaccion",
    "hhi", "share_fav", "total_transacciones"
]
df.drop(columns=[col for col in cols_a_eliminar if col in df.columns], inplace=True)

In [118]:
cols_a_eliminar = [
    "Unnamed: 0", "user_id", "total_spend", "monto_promedio_mensual.x", "monto_promedio_transaccion",
    "hhi", "share_fav", "total_transacciones"
]
df.drop(columns=[col for col in cols_a_eliminar if col in df.columns], inplace=True)

In [119]:
# >>> MODIFICACIÓN: Convertir columnas de fecha a numérico para regresión logística
# Ejemplo: convertir 'fecha_primer_producto' y 'fecha_segundo_producto'
fechas = ["fecha_primer_producto", "fecha_segundo_producto","mes_mas_compras.x", "mes_mayor_monto.x"]
for col in fechas:
    if col in df.columns:
        # Convertir a datetime
        df[col] = pd.to_datetime(df[col], errors='coerce')
        # Extraer el timestamp usando astype (para evitar el warning)
        df[col + "_ts"] = df[col].astype('int64')
        # Eliminar la columna original si no es necesaria
        df.drop(col, axis=1, inplace=True)

## Transformaciones númericas, logaritmicas y normalización

In [120]:
# Aplicar log1p a variables con outliers positivos
if 'total_spend_fav' in df.columns:
    df['total_spend_fav'] = np.log1p(df['total_spend_fav'])

if 'variacion_mensual_promedio_pct.x' in df.columns:
    df['variacion_mensual_promedio_pct.x'] = np.log1p(df['variacion_mensual_promedio_pct.x'])

# Winsorización de variación mensual promedio
if 'variacion_mensual_promedio.x' in df.columns:
    p01 = df['variacion_mensual_promedio.x'].quantile(0.01)
    p99 = df['variacion_mensual_promedio.x'].quantile(0.99)
    df['variacion_mensual_promedio.x'] = df['variacion_mensual_promedio.x'].clip(p01, p99)

# Estandarizar recencia para escala
if 'recencia_transaccion' in df.columns:
    df['recencia_transaccion'] = StandardScaler().fit_transform(df[['recencia_transaccion']])

# Asegurar que estas columnas binarias estén en formato booleano
binarias_explicit = ['checking_account', 'savings_account', 'credit_card', 'investment']
for col in binarias_explicit:
    if col in df.columns:
        df[col] = df[col].astype(bool)

# Escalar numéricas continuas (sin tocar las binarias)
numericas_continuas = [
    'age', 'dias_entre_productos', 'antiguedad_cliente', 'numero_productos',
    'entertainment_count', 'food_count', 'health_count', 'shopping_count',
    'supermarket_count', 'transport_count', 'travel_count', 'total_spend_fav',
    'transacciones_promedio_mensual.x', 'variacion_mensual_promedio.x',
    'variacion_mensual_promedio_pct.x', 'n_meses_activos', 'recencia_transaccion'
]
numericas_continuas = [col for col in numericas_continuas if col in df.columns and col not in binarias_explicit]
df[numericas_continuas] = StandardScaler().fit_transform(df[numericas_continuas])

## Transformacion One hot encoding

In [121]:
categoricas = [
    'income_range', 'risk_profile', 'occupation', 'age_range_sturges',
    'primer_producto', 'segundo_producto', 'combinacion_productos',
    'categoria_favorita_monto'
]
categoricas = [col for col in categoricas if col in df.columns]
df = pd.get_dummies(df, columns=categoricas, drop_first=True)

## Datasets

In [122]:
# Separar variables predictoras y objetivo
y = df['insurance']
X = df.drop(columns=['insurance'])

# Guardar
X.to_csv("../data/processed/X_logistic_reduced.csv", index=False)
y.to_csv("../data/processed/y_logistic_reduced.csv", index=False)